In [1]:
import numpy as np
import math
# import pandas

In [2]:
# from numpy import genfromtxt
coords = np.genfromtxt('data/coords.csv', delimiter=',',skip_header=1)
data_2d = coords[:6,:2]
data_3d = coords[6:,:]
print(data_2d)
print(data_3d)

[[ 990.  333.]
 [1003.  595.]
 [ 909.  755.]
 [1542.  667.]
 [1764.  453.]
 [1654.  307.]]
[[ 0.   9.2 14.6]
 [ 0.   9.2  9.2]
 [ 0.  11.9  6.5]
 [ 6.5  0.   6.5]
 [11.9  0.  11.9]
 [ 9.2  0.  14.6]]


In [3]:
# #Create Dataset
# data_2d = np.array([[990, 333],
#                       [1003, 595],
#                       [909, 755],
#                       [1542, 667],
#                       [1764, 453],
#                       [1654, 307]])

# data_3d = np.array([[0,9.2,14.6],
#                       [0,9.2,9.2],
#                       [0,11.9,6.5],
#                       [6.5,0,6.5],
#                       [11.9,0,11.9],
#                       [9.2,0,14.6]])

In [4]:
def get_avg_dist(points):
    d_avg = 0
    centroid = points.mean(0)
    for p in points:
        d_avg += np.linalg.norm(p - centroid)
    d_avg /= points.shape[0]
    return d_avg

def normalize_2d(points_2d):
    d_avg_2d = get_avg_dist(points_2d)
    centroid_2d = points_2d.mean(0)
    
    # Compute T matrix for normalizing 2D points
    T = np.array([[1,0,(-1)*centroid_2d[0]], [0,1,(-1)*centroid_2d[1]], [0,0,1]])
    T *= math.sqrt(2)/d_avg_2d
    T[2,2] = 1    
    print("T matrix for 2D normalization:\n" + str(T))
    
    # Find normalized 2D coordinates
    x = np.append(points_2d.T, np.array([[1,1,1,1,1,1]]), axis=0)
    x_norm = np.dot(T, x)
    points_2d_norm = x_norm[0:2].T
    print("Normalized 2D coordinates:\n" + str(points_2d_norm))
    return points_2d_norm, T

def normalize_3d(points_3d):
    d_avg_3d = get_avg_dist(points_3d)
    centroid_3d = points_3d.mean(0)
    
    # Compute U matrix for normalizing 3D points
    U = np.array([[1,0,0,(-1)*centroid_3d[0]], [0,1,0,(-1)*centroid_3d[1]], [0,0,1,(-1)*centroid_3d[2]], [0,0,0,1]])
    U *= math.sqrt(3)/d_avg_3d
    U[3,3] = 1  
    print("U matrix for 3D normalization:\n" + str(U))
    
    # Find normalized 3D coordinates
    X = np.append(points_3d.T, np.array([[1,1,1,1,1,1]]), axis=0)
    X_norm = np.dot(U, X)
    points_3d_norm = X_norm[0:3].T
    print("Normalized 3D coordinates:\n" + str(points_3d_norm))
    return points_3d_norm, U

def compute_P(points_3d, points_2d):
    # Form the P matrix
    points_3d_hom = np.append(points_3d, np.ones((6,1)), axis=1)
    P=np.empty((0,12))

    for i in range(points_3d.shape[0]):
        first = np.append(points_3d_hom[i], np.zeros(4))
        first = np.append(first, (-1)*points_2d[i][0]*points_3d_hom[i]) 
        first = first.reshape((1,12))
        P = np.append(P, first, axis=0)
        second = np.append(np.zeros(4), points_3d_hom[i])
        second = np.append(second, (-1)*points_2d[i][1]*points_3d_hom[i]) 
        second = second.reshape((1,12))
        P = np.append(P, second, axis=0)

    print("P matrix:\n" + str(P))  
    return P

def compute_m(P, T, U):
    # Find solution to Pm = 0 
    w, v = np.linalg.eig(np.dot(P.T,P))
    m = v[:,np.argmin(w)]
    M = m.real.reshape((3,4))    
    
    # Denormalize Projection Matrix m
    M = np.dot(M, U)
    M = np.dot(np.linalg.inv(T), M)
    print("Denormalized projection matrix m:\n" + str(M)) 
    return M

def recover_2d_points(points_2d,points_3d, M):
    # Recover 2D points using projection matrix found above
    points_3d_test = np.append(points_3d, np.ones((6,1)), axis=1)
    points_3d_test = points_3d_test.T
    points_2d_test = np.empty((0,2))

    for i in range(points_2d.shape[0]):
        x_i = np.dot(M[0], points_3d_test[:,i])/np.dot(M[2], points_3d_test[:,i])
        y_i = np.dot(M[1], points_3d_test[:,i])/np.dot(M[2], points_3d_test[:,i])
        points_2d_test = np.append(points_2d_test, np.array([[x_i,y_i]]), axis=0)

    print("Recovered 2D points:\n" + str(points_2d_test))
    return points_2d_test

def get_rmse(points_2d_test, points_2d):
    # Compute RMSE
    rmse = (points_2d_test - points_2d)**2
    rmse = rmse.mean(0)
    rmse = math.sqrt((rmse[0]+rmse[1])/2)
    print("Root mean squared Error: ", rmse)  
    return rmse

def get_intrinsic_parameters(M):
    # Compute Intrinsic Parameters
    A = M[:,0:3]
    b = M[:,3:4]
    epsilon = 1
    rho = epsilon/np.linalg.norm(A[2])
    x0 = rho*rho*np.dot(A[0],A[2])
    y0 = rho*rho*np.dot(A[1],A[2])
    print("Scaling Factor (rho): ", rho)
    print("x0: ", x0)
    print("y0: ", y0)

    a1_a3 = np.cross(A[0],A[2])
    a2_a3 = np.cross(A[1],A[2])
    cos_theta = (-1)*np.dot(a1_a3, a2_a3)/(np.linalg.norm(a1_a3)*np.linalg.norm(a2_a3))
    theta = math.acos(cos_theta)*180/math.pi
    sin_theta = math.sin(theta*math.pi/180)
    print("Theta: ", theta)
    print("Cos(theta): ", cos_theta)

    alpha = rho*rho*np.linalg.norm(a1_a3)*sin_theta
    beta = rho*rho*np.linalg.norm(a2_a3)*sin_theta
    print("Alpha: ", alpha)
    print("Beta: ", beta)

    K = np.array([[alpha, (-1)*alpha*cos_theta/sin_theta, x0], [0, beta/sin_theta, y0], [0, 0, 1]])
    print("K matrix:\n" + str(K))    
    return alpha, beta, x0, y0, theta, K

def get_extrinsic_parameters(M, K):
    # Compute Extrinsic parameters
    A = M[:,0:3]
    b = M[:,3:4]
    epsilon = 1
    rho = epsilon/np.linalg.norm(A[2])  
    a1_a3 = np.cross(A[0],A[2])
    a2_a3 = np.cross(A[1],A[2])    
    
    r3 = rho*A[2]
    r1 = a2_a3/np.linalg.norm(a2_a3)
    r2 = np.cross(r3, r1)
    R = np.empty((0,3))
    R = np.append(R, r1.reshape((1,3)), axis=0)
    R = np.append(R, r2.reshape((1,3)), axis=0)
    R = np.append(R, r3.reshape((1,3)), axis=0)
    print("R matrix:\n" + str(R))

    t = rho*np.dot(np.linalg.inv(K), b)
    print("t matrix:\n" + str(t))    
    
    return R, t

In [5]:
points_2d_norm, T = normalize_2d(data_2d)
points_3d_norm, U = normalize_3d(data_3d)

T matrix for 2D normalization:
[[ 3.70566783e-03  0.00000000e+00 -4.85566008e+00]
 [ 0.00000000e+00  3.70566783e-03 -1.92077116e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Normalized 2D coordinates:
[[-1.18704893 -0.68678377]
 [-1.13887525  0.2841012 ]
 [-1.48720802  0.87700805]
 [ 0.85847971  0.55090928]
 [ 1.68113797 -0.24210363]
 [ 1.27351451 -0.78313113]]
U matrix for 3D normalization:
[[ 0.22300884  0.          0.         -1.02584067]
 [ 0.          0.22300884  0.         -1.12619465]
 [ 0.          0.          0.22300884 -2.35274328]
 [ 0.          0.          0.          1.        ]]
Normalized 3D coordinates:
[[-1.02584067  0.92548669  0.90318581]
 [-1.02584067  0.92548669 -0.30106194]
 [-1.02584067  1.52761057 -0.90318581]
 [ 0.4237168  -1.12619465 -0.90318581]
 [ 1.62796455 -1.12619465  0.30106194]
 [ 1.02584067 -1.12619465  0.90318581]]


In [6]:
P = compute_P(points_3d_norm, points_2d_norm)

P matrix:
[[-1.02584067  0.92548669  0.90318581  1.          0.          0.
   0.          0.         -1.21772307  1.09859799  1.07212575  1.18704893]
 [ 0.          0.          0.          0.         -1.02584067  0.92548669
   0.90318581  1.         -0.70453073  0.63560924  0.62029336  0.68678377]
 [-1.02584067  0.92548669 -0.30106194  1.          0.          0.
   0.          0.         -1.16830455  1.05401389 -0.34287199  1.13887525]
 [ 0.          0.          0.          0.         -1.02584067  0.92548669
  -0.30106194  1.          0.29144257 -0.26293188  0.08553206 -0.2841012 ]
 [-1.02584067  1.52761057 -0.90318581  1.          0.          0.
   0.          0.         -1.52563848  2.27187469 -1.34322518  1.48720802]
 [ 0.          0.          0.          0.         -1.02584067  1.52761057
  -0.90318581  1.          0.89967053 -1.33972677  0.79210123 -0.87700805]
 [ 0.4237168  -1.12619465 -0.90318581  1.          0.          0.
   0.          0.         -0.36375228  0.96681526  0.7

In [7]:
M = compute_m(P, T, U)

Denormalized projection matrix m:
[[-1.71827428e+01  3.12075749e+01  4.75921141e+00 -9.81900974e+02]
 [-1.25426669e+00 -2.51010383e+00  3.26293545e+01 -6.63527845e+02]
 [ 5.77660173e-03  7.48176032e-03  3.24309813e-03 -7.47753556e-01]]


In [8]:
points_2d_test = recover_2d_points(data_2d,data_3d, M)
rmse = get_rmse(points_2d_test, data_2d)

Recovered 2D points:
[[ 990.07978675  332.87128297]
 [1002.96060133  595.34702437]
 [ 908.96984317  754.82548362]
 [1542.03249886  666.91737521]
 [1764.06504406  453.08499945]
 [1653.89241247  306.95338747]]
Root mean squared Error:  0.13208710504878887


In [9]:
np.savetxt("data/recovered_coords.csv", points_2d_test, delimiter=",", fmt='%i')

In [10]:
alpha, beta, x0, y0, theta, K = get_intrinsic_parameters(M)

Scaling Factor (rho):  100.06829463594272
x0:  1498.6881904672402
y0:  799.0383270464773
Theta:  88.8360402737667
Cos(theta):  0.020313532295953694
Alpha:  3268.8328518846265
Beta:  3177.6579553882384
K matrix:
[[ 3.26883285e+03 -6.64152459e+01  1.49868819e+03]
 [ 0.00000000e+00  3.17831377e+03  7.99038327e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
R, t = get_extrinsic_parameters(M, K)

R matrix:
[[-0.79479317  0.60666633  0.01611778]
 [-0.18481504 -0.26725222  0.94573762]
 [ 0.57805468  0.748687    0.3245313 ]]
t matrix:
[[  4.20523744]
 [ -2.07937932]
 [-74.82642318]]
